# Non-Maximum Suppression

https://d2l.ai/chapter_computer-vision/anchor.html#predicting-bounding-boxes-with-non-maximum-suppression

O **Non-Maximum Suppression (NMS)** é uma técnica utilizada para eliminar predições redundantes em detecção de objetos, mantendo apenas as mais relevantes. Em algoritmos de detecção, múltiplas bounding boxes podem se sobrepor ao detectar o mesmo objeto, gerando predições duplicadas. O NMS seleciona a bounding box com a maior pontuação de confiança e descarta as demais BBs cujo IoU com a bb principal exceda um limite predefinido. Isso ajuda a reduzir redundâncias e a garantir que cada objeto seja representado por uma única bounding box.

In [ ]:
import json
from src.standalone  import *
import numpy as np
from scipy.spatial.transform import Rotation as R

## Evaluate

Função evaluate que recebe duas bounding boxes no formato das predictions e retorna a métrica IoU 

In [4]:
def evaluate(prediction_instance1, prediction_instance2):
    center_prediction = prediction_instance1["center3D"]
    center = np.array([center_prediction["x"], center_prediction["y"], center_prediction["z"]])
    
    rotation_prediction = prediction_instance1["rotation3D"]
    rotation = [rotation_prediction["x"], rotation_prediction["y"], rotation_prediction["z"]]
    
    size_prediction = prediction_instance1["size3D"]
    size = [size_prediction["x"], size_prediction["y"], size_prediction["z"]]
    
    # Bounding Box Prediction 1
    p = center
    r = R.from_euler('xyz', rotation, degrees=True).as_matrix() # euler angles
    d = np.array(size) # size
    T = np.vstack([np.column_stack([d*r,p]),[0,0,0,1]])
    bb = OBB(T)

    center_prediction = prediction_instance2["center3D"]
    center = np.array([center_prediction["x"], center_prediction["y"], center_prediction["z"]])
    
    rotation_prediction = prediction_instance2["rotation3D"]
    rotation = [rotation_prediction["x"], rotation_prediction["y"], rotation_prediction["z"]]
    
    size_prediction = prediction_instance2["size3D"]
    size = [size_prediction["x"], size_prediction["y"], size_prediction["z"]]
    
    # Bounding Box Prediction 2
    p = center
    r = R.from_euler('xyz', rotation, degrees=True).as_matrix() # euler angles
    d = np.array(size) # size
    T = np.vstack([np.column_stack([d*r,p]),[0,0,0,1]])
    bb2 = OBB(T)

    # IoU
    iou_res = bb.IoU_v(bb2,1e-8)

    return iou_res

## Data

In [5]:
import os, json

folder_path = 'results/results'
pred_names = os.listdir(folder_path)

predictions = {}

for pred_name in pred_names:
    
    if "checkpoints" in pred_name:
        continue
        
    predictions[pred_name] = []
    
    with open(f"{folder_path}/{pred_name}", 'r') as file:
        pred_json = json.load(file)
        pred_signals = pred_json["signalsReceived"]

        for detections in pred_signals.values():
            for pred in detections["detections"]:
                predictions[pred_name].append(pred)

## NMS

In [6]:
import torch

def nms_3d_predictions(predictions, threshold=0.5):
    # Ordena as predicoes de um arquivo pela confianca em ordem descendente
    predictions = sorted(predictions, key=lambda x: x['confidence'], reverse=True)
    keep = []

    while predictions:
        # Seleciona a predicao com a confianca mais alta
        best_prediction = predictions.pop(0)
        
        keep.append(best_prediction)
        
        if not predictions:
            break

        # Calcula o iou entre a melhor predicao e todas as outras
        remaining_preds = []
        for pred in predictions:
            iou = evaluate(best_prediction, pred)
            
            # So mantem as predicoes cujo IoU é menor que o threshold
            if iou <= threshold:
                remaining_preds.append(pred)
        
        # Atualiza a lista com as predicoes remanescentes
        predictions = remaining_preds

    return keep

In [ ]:
for pred_name in predictions:
    print(pred_name)
    predictions_set = predictions[pred_name]
    filtered_predictions = nms_3d_predictions(predictions_set)
    print(f"{len(predictions_set)} -> {len(filtered_predictions)}")
    print(f"{len(predictions_set) - len(filtered_predictions)} filtrados\n")